# Run Test 3.30.23
- 9 days (465 x 9 x 7000= 29M total FFTs, 2^24, 660MHz clock) 
- switch antenna and term (switch=1 is terminator. 1/16 duty cycle) 
- 4dB fixed attenuation, pasternack 1012 to mini circuits zkl1r5+ (9V regulated from battery). 
- Run started 4-19-23 at 16:19:15. 
- Right at clipping

In [ ]:
import sys 
import numpy as np
import settings as s
import matplotlib.pyplot as plt 
import importlib
importlib.reload(s)
from cupyx.profiler import benchmark
import h5py
import os
import re
import time
import datetime
from datetime import datetime
import pandas as pd

sys.path.insert(0, '../../../drlib')
import drlib as dr
%matplotlib widget

def mad_np(arr):
    median = np.median(arr)
    return np.median(np.abs(arr-median))

def freq2Idx(freq, freqsAll):
    return int(freq/freqsAll[-1] * len(freqsAll))

def idx2freq(idx, freqs):
    return  ((freqs[-1] - freqs[0])/len(freqs) * idx) + freqs[0]

dataDir         = '/drBiggerBoy/run1p3/main_data/'
filename        = 'packedh5.hdf5'
databaseFile    = 'database.txt'

totalAcqs       = int(open(s.SAVE_DIRECTORY + databaseFile, 'r'
                    ).readlines()[-1].split(',')[0].strip()) + 1 

print(totalAcqs)
    
# Generate antIdx and termIdx from database file
# Open the data file and read the lines
with open(dataDir + databaseFile, 'r') as f:
    # Skip the header line
    next(f)
    # Initialize the two arrays
    antIdx = []
    termIdx = []
    # Loop through the remaining lines and extract the acqNum values
    for line in f:
        # Split the line into fields
        fields = line.strip().split(',')
        # Extract the SWITCH_POS and ACQ NUM values
        switch_pos = int(fields[2])
        acq_num = int(fields[0])
        # Append the acqNum value to the appropriate array
        if switch_pos == 0:
            antIdx.append(acq_num)
        elif switch_pos == 1:
            termIdx.append(acq_num)
    
    #find acqNums that are within first three after antenna move
    badAcqs = [0,1,2]
    for i in range(1, totalAcqs):
        if i % 464 == 0:
            badAcqs.append(i)
            badAcqs.append(i)
            badAcqs.append(i+2)
    print(badAcqs)

#Throw out three after antenna move and everything after 1855 (last in run 3)
antIdx  = [i for i in antIdx if i not in(badAcqs) and i < 1855]
termIdx = [i for i in termIdx if i not in(badAcqs) and i < 1855
           ]

# Print the results
print("antIdx =", antIdx)
print("termIdx =", termIdx)

for i in antIdx:
    print(i)


In [ ]:
#time delta analysis
datetimes_str = np.loadtxt(f'{dataDir}database.txt',
                        skiprows=1,
                        usecols=1,
                        delimiter=',',
                        dtype=str)
datetimes =[]
for date in datetimes_str:
    data = date[10]
    datetimes.append(datetime.strptime(date, ' %Y-%m-%d %H:%M:%S.%f'))


diff = []
for i in range(len(datetimes)-1):
    diff.append((datetimes[i+1] - datetimes[i]).total_seconds())

print(f"diff mean:              {np.mean(diff)}")
print(f"diff median:            {np.median(diff)}")
print(f"diff STD:               {np.std(diff)}")
print(f"diff percent error:     {np.std(diff)/np.mean(diff) * 100}")
print(f"expected single time:   {2**24/s.SAMPLE_RATE * 250}")
print(f"total time:             {(datetimes[-1] - datetimes[0]).total_seconds()}")
print(f"expected total time:    {2**24/s.SAMPLE_RATE * s.NOF_BUFFERS_TO_RECEIVE * s.NOF_ACQUISITIONS_TO_TAKE}")

plt.close('all')
plt.figure()
plt.plot(datetimes[1:], diff)
plt.xlabel('Acquisition Index')
plt.ylabel('Time Delta (Seconds)')

In [ ]:
#pack pre proc dataset

try:
    f.close()
    print("closed")
except:
    print("didn't close")
    pass

rawDataDir      = dataDir + 'data/'
fileList        = os.listdir(rawDataDir)
fileList.sort(key=lambda f: int(re.sub('\D', '', f)))
newDataDir      = dataDir
fileName        = 'packedh5.hdf5'
if 1: #will overwrite!!!

    f               = h5py.File(newDataDir + fileName, 'w')

    #init empty dataset
    print(f'initilizing dataset 8388609x{totalAcqs}')
    specA_W    = f.create_dataset('spec_W_chA', (8388609, totalAcqs), chunks = (2**16,2**0), dtype = 'f')
    specB_W    = f.create_dataset('spec_W_chB', (8388609, totalAcqs), chunks = (2**16,2**0), dtype = 'f')
    print(specA_W)
    print(specB_W)

    ##############################################################
    #Main loop
    ##############################################################
    numFiles        = 0
    numMeasData     = 0
    numRigolSpec    = 0

    compType = False
    chunks = None #must add chunks arg to create dataset calls

    for fileidx, file in enumerate(fileList):
        if fileidx % 1 == 0:
            print(file)
        dataset         = h5py.File(rawDataDir +  file, 'r')
        numFiles        += 1
        measDataKeys    = list(dataset.keys())
        measDataKeys.sort(key=lambda f: int(re.sub('\D', '', f)))
        #measDataKeys = [int(key) for key in measDataKeys]

        for measData in measDataKeys:
            print(measData)
            measDataInt = int(measData)
            '''Dumb check. Is the date in raw h5 measData the same as what
            the database thinks it is? Since we previously check that dates
            are consectutive this also checks that dates 
            in raw h5 are consecutive'''
            '''datasetDf   = pd.read_hdf(rawDataDir + file, key = measData)#raw H5 
            dateTimeStr = datasetDf.columns[0][0]#datetime extracted from raw H5
            dateTime    = datetime.strptime(dateTimeStr, '%Y-%m-%d %H:%M:%S.%f')'''

            '''Check database at a measData has the same datetime as we are about
            to load into new h5'''
            '''if not (dateTime==database[database['measData'] == measDataInt].index):
                print('Dates got jumbled at measData', measDataInt)'''
            specA_W_np        = np.float32(dataset[measData]['chASpec_W'])
            specB_W_np        = np.float32(dataset[measData]['chBSpec_W'])


            specA_W[:, measDataInt]    = specA_W_np
            specB_W[:, measDataInt]    = specB_W_np

f.close()

In [ ]:

f               = h5py.File(dataDir + fileName, 'r')
vetoSpec = f['spec_W_chA'][:,1]
antSpec  = f['spec_W_chB'][:, 1+(4*464)]


freqasdf = np.linspace(0,330,2**23)
plt.close('all')
plt.figure()
#plt.plot(freqasdf, 10*np.log10(vetoSpec[1:]*1000))

termToPlt = [1,
             1841,
            1+(4*464),
            1873,
            1905,
            2001,
            2305]
pltLabels = ['Acq. 1',
             'Acq. 1841 (Last in run 3)',
             'Acq. 1857 (First in run 4)',
             'Acq. 1905',
             'Acq. 2001',
             'Acq. 2305 (Last in run 4)']

print('term idxs to plot', termToPlt)

for label, idx in zip (pltLabels, termToPlt):
    plt.plot(freqasdf, 10*np.log10(f['spec_W_chB'][1:, idx]*1000),
             label = label
             )

plt.title('Terminator Spectrum')
plt.legend(loc = 'upper right')
plt.xlabel('Freq (MHz)')
plt.ylabel('Power (dBm)')
f.close()


In [ ]:
f               = h5py.File(dataDir + fileName, 'r')
avgPow          = []
deadTermIdxs    = np.arange(1,2306,16)
for i in deadTermIdxs:
    avgPow.append(f['spec_W_chB'][100000::10000, i].mean())
f.close()

plt.close('all')
plt.title('Avg. Power vs Time')
plt.scatter(deadTermIdxs,10*np.log10((avgPow) ) + 30)
plt.xlabel('Acq. Num. (Linear in Time)')
plt.ylabel('Power (dBm) Not Gain Corrected')
plt.show()

plt.figure()
plt.title('Avg. Power vs Time')
plt.scatter(deadTermIdxs, avgPow)
plt.xlabel('Acq. Num. (Linear in Time)')
plt.ylabel('Power (W) Not Gain Corrected')
plt.show()

In [ ]:
f               = h5py.File(dataDir + fileName, 'r')
print(f.keys())
print(f['spec_W_chA'][0,:])

f.close()

In [ ]:
#Average and save spectra

#location of packed h5
fileName        = 'packedh5.hdf5'




def avgSpecFromPacked(avgIdxs, specStr):
    sum = np.zeros(2**23+1)
    for i in avgIdxs:
        sum += f[specStr][:,i]
    avgSpec = sum/(len(avgIdxs))
    return avgSpec

def rollingMadLim(spec,
    nSigma = 1,
    order = 1,
    fc_numBins = 100
):
    specFiltered = dr.filterSpec(spec, fc_numBins=fc_numBins, order=order)
    window = 1000

    rollingMadArr = dr.rolling(specFiltered, window, 1, mad_np, numProc=48)

    filter_fc           = 30 * window
    filteredMadArr  = dr.filterSpec(rollingMadArr, order = 2, fc_numBins = filter_fc, type = 'lowpass')
    diffFilterMadPadArr = dr.nanPad(filteredMadArr, window)
    rollMadLim      = diffFilterMadPadArr * nSigma * 1.48
    return rollMadLim, specFiltered, nSigma, order, fc_numBins

#Average, filter and save
if 0:
    f               = h5py.File(dataDir + fileName, 'r')
    
    #antIdxs     = range(0, totalAcqs-1, 2)[0:-1]
    #termIdxs    = range(1, totalAcqs-1, 2)
    avgAntSpec  = avgSpecFromPacked(antIdx, 'spec_W_chB')
    avgVetoSpec  = avgSpecFromPacked(antIdx, 'spec_W_chA')
    avgTermSpec = avgSpecFromPacked(termIdx, 'spec_W_chB')

    np.save('./avgSpecs_gitignore/avgAntSpec_W_12e6FFTs_5_19_23', avgAntSpec)
    np.save('./avgSpecs_gitignore/avgVetoSpec_W_12e6FFTs_5_19_23', avgVetoSpec)
    np.save('./avgSpecs_gitignore/avgTermSpec_W_8e5FFTs_5_19_23', avgTermSpec)
    print('Done saving raw spectra')

    #filter/rolling mad limit and save
    antRollingMadLim, antFilt, Sigma, order, fc_numBins     = rollingMadLim(avgAntSpec)
    termRollingMadLim, termFilt, nSigma, order, fc_numBins   = rollingMadLim(avgTermSpec)

    np.save(f'./avgSpecs_gitignore/antRollingMadLim_W_12p6e6FFTs_nSigma{nSigma}_order{order}_fc{fc_numBins}_4_24_23', antRollingMadLim)
    np.save(f'./avgSpecs_gitignore/termRollingMadLim_W_8e5FFTs_nSigma{nSigma}_order{order}_fc{fc_numBins}_4_24_23', termRollingMadLim)

    np.save(f'./avgSpecs_gitignore/antFilt_W_12e6FFTs_nSigma{nSigma}_order{order}_fc{fc_numBins}_5_19_23', antFilt)
    np.save(f'./avgSpecs_gitignore/termFilt_W_8e5FFTs_nSigma{nSigma}_order{order}_fc{fc_numBins}_5_19_23', termFilt)

    f.close()


In [ ]:
#load files generated in previous cell
# only saved nSigma=1, changing nSigma just multplies the rolling mad limit in this cell
nSigma      = 5
order       = 1
fc_numBins  = 100

avgAntSpec          = np.load('./avgSpecs_gitignore/avgAntSpec_W_12e6FFTs_5_19_23.npy', allow_pickle=True)
avgTermSpec         = np.load('./avgSpecs_gitignore/avgTermSpec_W_8e5FFTs_5_19_23.npy', allow_pickle=True)
avgVetoSpec         = np.load('./avgSpecs_gitignore/avgVetoSpec_W_12e6FFTs_5_19_23.npy', allow_pickle=True)

filtAntSpec         = np.load(f'./avgSpecs_gitignore/antFilt_W_12e6FFTs_nSigma1_order{order}_fc{fc_numBins}_5_19_23.npy', 
    allow_pickle=True
)
filtTermSpec        = np.load(f'./avgSpecs_gitignore/termFilt_W_8e5FFTs_nSigma1_order{order}_fc{fc_numBins}_5_19_23.npy', 
    allow_pickle=True
)
termRollingMadLim   = nSigma * np.load(f'./avgSpecs_gitignore/termRollingMadLim_W_8e5FFTs_nSigma5_order{order}_fc{fc_numBins}_5_19_23.npy',
    allow_pickle=True
)
antRollingMadLim   = nSigma * np.load(f'./avgSpecs_gitignore/antRollingMadLim_W_12p6e6FFTs_nSigma5_order{order}_fc{fc_numBins}_5_19_23.npy',
    allow_pickle=True
)


#plot raw spectra
freqs       = np.linspace(0, 330e6, 2**23, dtype=np.float64)

plt.close('all')

plt.title('Raw Spectra (Watts)')
plt.plot(freqs/1e6, avgTermSpec[1:], label = 'Terminator')
plt.plot(freqs/1e6, avgAntSpec[1:], label = 'Amtenna')
plt.ylabel('Power (Watts)')
plt.xlabel('Freqency (MHz)')
plt.legend(loc = 'upper right')

plt.figure()
plt.title('Raw Spectra (dBm)')
plt.plot(freqs/1e6, 10*np.log10(avgTermSpec[1:]*1000), label = 'Term')
plt.plot(freqs/1e6, 10*np.log10(avgAntSpec[1:]*1000), label = 'Antenna')
plt.ylabel('Power (dBm)')
plt.xlabel('Freqency (MHz)')
plt.legend(loc = 'upper right')

plt.figure()
plt.title('Veto Spectrum (dBm)')
plt.plot(freqs/1e6, 10*np.log10(avgVetoSpec[1:]*1000))
plt.ylabel('Power (dBm)')
plt.xlabel('Freqency (MHz)')



In [ ]:
print(f'nSigma: {nSigma}')

pltFreqStart = 45e6
pltFreqStop = 305e6
pltIdxStart = freq2Idx(pltFreqStart, freqs)
pltIdxStop = freq2Idx(pltFreqStop, freqs)

antCanList_idx = ((filtAntSpec > antRollingMadLim)) 
antCandidateTotal = (antCanList_idx[pltIdxStart:pltIdxStop].sum())
print(f"{antCandidateTotal} antenna candidates")

termCanList_idx = ((filtTermSpec > termRollingMadLim)) 
termCandidateTotal = (termCanList_idx[pltIdxStart:pltIdxStop].sum())
print(f"{termCandidateTotal} term candidates")


plt.close('all')

plt.figure()
plt.plot(freqs[pltIdxStart:pltIdxStop]/1e6, filtAntSpec[pltIdxStart:pltIdxStop])
plt.plot(freqs[pltIdxStart:pltIdxStop]/1e6,(antRollingMadLim)[pltIdxStart:pltIdxStop], label = f'{nSigma} Sigma')
plt.text(200,.75e-14, f'{antCandidateTotal} candidates',size=16)
plt.title(f"Antenna Filtered Linear Power from {pltFreqStart/1e6} to {pltFreqStop/1e6}Mhz")
plt.xlabel('Frequency (MHz)')
plt.ylabel('Filtered Watts')
plt.legend(loc='upper right')

plt.figure()
plt.plot(freqs[pltIdxStart:pltIdxStop]/1e6, filtTermSpec[pltIdxStart:pltIdxStop])
plt.plot(freqs[pltIdxStart:pltIdxStop]/1e6,(termRollingMadLim)[pltIdxStart:pltIdxStop],label = f'{nSigma} Sigma')
plt.text(200,.75e-14, f'{termCandidateTotal} candidates',size=16)
plt.title(f"Terminator Filtered Linear Power from {pltFreqStart/1e6} to {pltFreqStop/1e6}Mhz")
plt.xlabel('Frequency (MHz)')
plt.ylabel('Filtered Watts')
plt.legend(loc='upper right')

if 0:
    plt.figure()
    plt.plot(freqs[pltIdxStart:pltIdxStop]/1e6, filtAntSpec[pltIdxStart:pltIdxStop], label = 'Antenna')
    plt.plot(freqs[pltIdxStart:pltIdxStop]/1e6, filtTermSpec[pltIdxStart:pltIdxStop], label = 'Terminator', alpha = .5)
    plt.text(200,.75e-14, f'{termCandidateTotal} candidates',size=16)
    plt.title(f"Filtered Linear Power from {pltFreqStart/1e6} to {pltFreqStop/1e6}Mhz")
    plt.xlabel('Frequency (MHz)')
    plt.ylabel('Filtered Watts')
    plt.legend(loc = 'upper right')

In [ ]:
#generate lists of candidates

antCanIdx   = (np.where(antCanList_idx  == True))[0]
termCanIdx  = (np.where(termCanList_idx == True))[0]

antMask     = (antCanIdx >= pltIdxStart) & (antCanIdx <= pltIdxStop)
termMask    = (termCanIdx >= pltIdxStart) & (termCanIdx <= pltIdxStop)

antCanIdx   = antCanIdx[antMask]
termCanIdx  = termCanIdx[termMask]

antCanFreqs =np.asarray([idx2freq(idx,freqs) for idx in antCanIdx])
termCanFreqs=np.asarray([idx2freq(idx,freqs) for idx in termCanIdx])



print(len(antCanFreqs), 'antenna candidates')
#print('at freqs. ', antCanFreqs/1e6)
print(len(termCanFreqs), 'terminator candidates')
print('at freqs. ', termCanFreqs/1e6)

In [ ]:
def consolidateCandidates(rawCanList, freqs, stepsize=1):
    '''
    Takes array of indices of candidates and returns a data frame
    with one row per candidate and columns with the candidate frequency,
    number of bins wide and the starting index of that candidate
    Inputs:
        rawCanList (1D list or np array): list of indices of candidates
        reqs (1D list or np array): list of frequencies
        stepsize: should be 1 to find adjecent candidates
    Calls:
        idx2Freq()
    '''
    rawCanList = np.asarray(rawCanList)
    splitList =  np.split(rawCanList, np.where(np.diff(rawCanList) > stepsize)[0]+1)
    canList = [can[0] for can in splitList]
    canListFreq =  [idx2freq(can[0],freqs) for can in splitList]
    canLenList = [len(can) for can in splitList]
    candidate_df = pd.DataFrame()
    candidate_df['canFreq_Mhz'] = [freq/1e6 for freq in canListFreq]
    candidate_df['numBins'] = canLenList
    candidate_df['idx'] = canList
    #candidate_df['idx absolute'] = [idx + idxStart for idx in canList]
    diffArr = np.zeros(len(canList))
    diffArr[0] =  float('nan')
    diffArr[1:] = np.diff(canList)
    candidate_df['deltaIdx'] = diffArr
    candidate_df['deltaFreq_Hz'] = [deltaIdx* (s.SAMPLE_RATE) /s.CH0_RECORD_LEN for deltaIdx in diffArr]

    return candidate_df

stepsize = 10
candidate_df = consolidateCandidates(antCanIdx, freqs, stepsize=stepsize)
#candidate_df.to_csv(f'./candidateListCsvs/candidateList_run1_firstThird_diffGreaterThan10Bins_{nSigma}sigma_fc100bins_Order1_4_24_23.csv')
candidate_df

In [ ]:

fig, ax = plt.subplots()

ax.plot(np.arange(0, len(freqs), 1), np.zeros(len(freqs)), alpha = 0)
ax.set_xlabel('bin number')
ax.xaxis.tick_top()
ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
ax.xaxis.set_ticks_position('top')
ax.set_xlim(pltIdxStart, pltIdxStop)
ax.set_ylim(np.min(filtAntSpec[pltIdxStart:pltIdxStop]), 
    np.max(filtAntSpec[pltIdxStart:pltIdxStop]*1.05)
)
ax.set_ylabel('Linear Power')

ax2 = ax.twiny()

ax2.plot(freqs/1e6, filtAntSpec[1:])
ax2.plot(freqs/1e6,  antRollingMadLim[1:], label = f'{nSigma} sigma (rolling mad)')
ax2.set_xlabel('Frequency (MHz)')
ax2.set_ylabel('Linear Power (W)')


#ax2.plot(np.arange(pltIdxStart, pltIdxStop, 1), antRollingMadLim[pltIdxStart:pltIdxStop], 'orange', label = '5 sigma (rolling mad)')
ax2.scatter(freqs[candidate_df['idx']]/1e6, antRollingMadLim[candidate_df['idx']], color ='r', marker = 'x')
ax2.set_xlim(pltFreqStart/1e6, pltFreqStop/1e6)
#ax2.set_ylim(np.min(filtAntSpec[pltIdxStart:pltIdxStop]), 
#    np.max(filtAntSpec[pltIdxStart:pltIdxStop])
#)

ax2.legend(loc = 'upper right')
ax2.xaxis.tick_top()

plt.show()

print('stepsize = %i ' %stepsize)
print('butter filter (order = %i and' %order, 'f_c = %i'  %fc_numBins, 'bins)')

print('number of candidates = ', len(candidate_df['idx']) )

In [ ]:
#create a list of bunches of spurs
print()
idxDiff = np.diff(antCanIdx)
#print(idxDiff)
diffMask = np.where(idxDiff == 1, True, False)
diffMask = np.append(False, diffMask,)

onesIdxs = antCanIdx[diffMask]
#print(freqs[onesIdxs])

hist, binEdges = np.histogram(idxDiff, bins = 10000, range=(1,10001))
#print(binEdges)
histNoZeros = []
binsNoZeros = []
for i, val in enumerate(hist):
    if val != 0 and val != 1:
        histNoZeros.append(val)
        binsNoZeros.append(binEdges[i])
binsNoZeros =[str(int(bin)) for bin in binsNoZeros]

count = 0
for val in idxDiff:
    if int(val) > 10000:
        count += 1
print("Count of differences greater than 10000: ", count)


plt.close('all')
plt.figure()
#plt.semilogy()
plt.scatter(np.arange(0, len(idxDiff), 1),(idxDiff))
plt.title('Difference Between Adjacent Candidates')
plt.xlabel('Index Within Candidate List')
plt.ylabel('Difference')
plt.grid()
plt.tight_layout()

#hist without skipping empty bins. Not useful
if 0:
    plt.figure();
    plt.hist(idxDiff, bins = 10000, range=(2,10002));
    plt.title('Difference Between Adjacent Candidates')
    plt.xlabel('Difference')
    plt.ylabel('Counts')

#hist skipping unused bins
plt.figure();
plt.bar(binsNoZeros, histNoZeros);
plt.title('Difference Between Adjacent Candidates');
plt.xlabel('Difference');
plt.ylabel('Counts');
plt.xticks(rotation='vertical');
plt.tight_layout()
plt.grid()

In [ ]:
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots
from plotly_resampler import FigureResampler, FigureWidgetResampler

x = freqs[pltIdxStart:pltIdxStop]
main_data = filtAntSpec[pltIdxStart:pltIdxStop]
sigmaLine = antRollingMadLim[pltIdxStart:pltIdxStop]
veto_data = avgVetoSpec[pltIdxStart:pltIdxStop]

# Create the subplots with shared x-axis
fig = FigureWidgetResampler(make_subplots(rows=2, cols=1, shared_xaxes=True),  default_n_shown_samples=10000)

# Add the main data trace to the first subplot
fig.add_trace(
    go.Scattergl(
        x=x,
        y=main_data,
        mode='lines',
        marker=dict(
            color='blue',
            size=1
        ),
        name='Main Channel, filtered Watts'
    ),
    row=1, col=1
)

# Add the sigma limit trace to the first subplot
fig.add_trace(
    go.Scattergl(
        x=x,
        y=sigmaLine,
        mode='lines',
        marker=dict(
            color='orange',
            size=1
        ),
        name='6 sigma limit'
    ),
    row=1, col=1
)
 #add an x where main channel power goes over 6 sigma
fig.add_trace(go.Scatter(
    x=freqs[candidate_df['idx']],
    y=antRollingMadLim[candidate_df['idx']],
    mode='markers',
    marker=dict(color='red', symbol='x'),
    name = 'Main power > 6 sigma'
))
 

# Add the veto data trace to the second subplot
fig.add_trace(
    go.Scattergl(
        x=x,
        y=10 * np.log10(veto_data * 1000),
        #y = veto_data,
        mode='lines',
        marker=dict(
            color='red',
            size=1
        ),
        name='Veto Channel'
    ),
    row=2, col=1
)

# Set the subplot titles and axis labels
fig.update_layout(
    title='Main and Veto Channels',
    xaxis_title='Frequency (MHz)',
    yaxis_title='Filtered Main Channel (Watts)',
    yaxis2_title='Veto (dBm)',
    yaxis=dict(exponentformat='power')
)

# Wrap the figure with FigureWidgetRescaler for resampling
#resampler = FigureWidgetRescaler(fig)

# Display the resampled figure
#resampler.widget()
